In [1]:
# Check core SDK version number
import azureml.core
print("SDK version:", azureml.core.VERSION)


SDK version: 1.51.0


In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')


mlv2
ml_group
eastasia
956a5d16-ed62-4076-890a-4f7ea58eef95


In [12]:
from azureml.core.model import Model
# Tip: When model_path is set to a directory, you can use the child_paths parameter to include
#      only some of the files from the directory
model = Model.register(model_path = "network.h5",
                       model_name = "operation_model_nb",
                       workspace = ws)

model = Model.register(model_path = "scale.pkl",
                       model_name = "scale",
                       workspace = ws)


Registering model operation_model_nb
Registering model scale


In [3]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
myenv = Environment(name="myenv")
conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
conda_dep.add_conda_package("joblib==1.2.0")
# conda_dep.add_conda_package("keras==2.11.0")
conda_dep.add_conda_package("numpy==1.24.2")
conda_dep.add_conda_package("pandas==1.5.3")
# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("scikit-learn==1.2.1")
conda_dep.add_pip_package("scipy==1.10.1")
conda_dep.add_pip_package("tensorflow==2.11.0")
# conda_dep.add_pip_package("xgboost==1.7.4")


# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)


In [12]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model
from azureml.core.compute.aks import AksCompute
from azureml.core.compute import ComputeTarget

aks_target = AksCompute(ws,"dev-aks2-v1")
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

network = Model(ws, name='operation_model_nb')
scale = Model(ws, name='scale')

service = Model.deploy(ws, 'operation-service-v2', [network,scale], inference_config, deployment_config, aks_target)

service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())


/tmp/ipykernel_5561/3503029620.py:12: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, 'operation-service-v2', [network,scale], inference_config, deployment_config, aks_target)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-08-16 14:11:58+00:00 Creating Container Registry if not exists.
2023-08-16 14:12:00+00:00 Use the existing image.
2023-08-16 14:12:02+00:00 Creating resources in AKS..
2023-08-16 14:12:03+00:00 Submitting deployment to compute..
2023-08-16 14:12:45+00:00 Checking the status of inference endpoint operation-service-v2.
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy
/bin/bash: /azureml-envs/azureml_d137506b3aca8565fa06cc9062edccec/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_d137506b3aca8565fa06cc9062edccec/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_d137506b3aca8565fa06cc9062edccec/lib/libtinfo.so.6: no version information available (requ